# Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On va entrainer un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

## 1. Création du jeu de données (*dataset*)

In [5]:
import json
import spacy
import numpy as np
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np
import pandas as pd

/Users/younismbs/anaconda3/envs/myEmv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train_json_fn = "./data/incidents_train.json"
validation_json_fn = "./data/incidents_test.json"
test_json_fn = "./data/incidents_test.json"

In [7]:
#Fonction permettant de charger les données
def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
    return incident_list

In [8]:
train_list = load_incident_dataset(train_json_fn)
validation_list = load_incident_dataset(validation_json_fn)
test_list = load_incident_dataset(test_json_fn)

print("Nombre d'incidents dans train:", len(train_list))
print("Nombre d'incidents dans validation:", len(validation_list))
print("Nombre d'incidents dans test:", len(test_list))

Nombre d'incidents dans train: 2475
Nombre d'incidents dans validation: 531
Nombre d'incidents dans test: 531


In [9]:
#On divise nos listes en X(texte) et y(labels) pour chacun des sets

X_train = [instance["text"] for instance in train_list]
y_train = [instance["label"] for instance in train_list]

X_val = [instance["text"] for instance in validation_list]
y_val = [instance["label"] for instance in validation_list]

X_test = [instance["text"] for instance in test_list]
y_test = [instance["label"] for instance in test_list]

In [10]:
nb_classes = len(set(y_train))
nb_classes

9

## 2. Gestion de plongements de mots (*embeddings*)

In [11]:
#On telecharge nos embeddings/tokenizer spacy
spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 528.4 kB/s eta 0:00:0000:0100:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [12]:
nlp = spacy.load('en_core_web_md')

## 3. Création de modèle(s)

In [13]:
class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size) :
        super().__init__()
        # Définition de la couche d'entrée à la couche cachée
        self.intput_layer = nn.Linear(input_size, hidden_layer_size) #On a une couche cachée de taille hidden_layer_size
        # Définition la couche cachée à la couche de sortie
        self.output_layer = nn.Linear(hidden_layer_size, output_size)
        
    # Fonction forward permettant la propagation avant à travers la couche cachée avec une fonction d'activation ReLU entre les deux couches.
    def forward(self, x):
        x = self.intput_layer(x)
        x = nn.functional.relu_(x)
        x = self.output_layer(x)
        return x

## 4. Fonctions utilitaires

Dans cette section on regroupe les differentes fonctions permettant de gerer l'agrégation des plongements de mots.

Le processus de ces differentes fonctions sont presque identiques, la seul difference va resider dans la maniere dont on va "fusionner" nos plongements de mot.

In [15]:
def average_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)  # Utilisation de notre tokenizer spaCy pour tokeniser la phrase en mots.
    nb_column = len(tokenised_sentence) 
    nb_rows =  nlp_model.vocab.vectors_length # Récupération de la taille des embeddings de mots de spaCy (300).
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column)) 
    # Ce qu'on fait dans cette boucle, c'est qu'on itère sur chacun des mots de notre phrase et on récupère son embedding correspondant pour l'agréger à notre matrice en tant que colonne.
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    # À la fin, une fois qu'on a notre matrice complète, on fait la moyenne des colonnes (axis = 1) pour obtenir un vecteur qui sera donné en entrée à notre réseau.
    return np.average(sentence_embedding_matrix, axis=1)

def maxpool_embedding(sentence, nlp_model=nlp): 
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.vocab.vectors_length 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                    
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    # On fait un max pooling sur les colonnes pour récupérer la valeur maximale.
    return np.max(sentence_embedding_matrix, axis=1)

def minpool_embedding(sentence, nlp_model=nlp): 
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows =  nlp_model.vocab.vectors_length 
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))                                    
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    # On fait un min pooling sur les colonnes pour récupérer la valeur minimale.
    return np.min(sentence_embedding_matrix, axis=1)

In [16]:
aggregations = {
    "average" : average_embedding,
    "maxpool" : maxpool_embedding,
    "minpool" : minpool_embedding
}

Ici on definit une classe Dataset permettant de construire notre dataset en fonction des listes de phrases et des classes correspondantes, en utilisant la fonction d'agrégation définie précédemment.

Notre dataset sera par la suite donnée a un dataloader qui se chargera de diviser notre dataset en batch

In [17]:
class SpacyDataset(Dataset):
    def __init__(self, dataset: List[str], target: np.array, sentence_aggregation_function):
        self.dataset = dataset
        self.doc_embeddings = [None for _ in range(len(dataset))]
        self.sentence_aggregation_function = sentence_aggregation_function 
        self.target = target
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.doc_embeddings[index] is None:
            # Si l'embedding n'est pas encore défini pour l'index correspondant, alors on le construit en appelant notre fonction d'agrégation.
            self.doc_embeddings[index] = self.sentence_aggregation_function(self.dataset[index]) 
        return FloatTensor(self.doc_embeddings[index]), LongTensor([int(self.target[index])]).squeeze(0)

## 5. Entraînement de modèle(s)

In [49]:
def train(nlp_model=nlp, aggregations=aggregations, hidden_size=100):
    # Initialisation de dictionnaires pour stocker les modèles, les journaux (loggings) et les expériences pour chaque type d'agrégation.
    models = {"average": None, "maxpool": None, "minpool": None}
    loggings = {"average": None, "maxpool": None, "minpool": None}
    experiments = {"average": None, "maxpool": None, "minpool": None}
    test_dataloaders = {"average": None, "maxpool": None, "minpool": None}
    
    # Itération sur chaque type d'agrégation (moyenne, maxpool, minpool).
    for key in aggregations.keys():
        # Création des datasets pour l'entraînement, la validation et le test en utilisant l'agrégation courante.
        train_dataset = SpacyDataset(X_train, y_train, aggregations[key])
        valid_dataset = SpacyDataset(X_val, y_val, aggregations[key])
        test_dataset = SpacyDataset(X_test, y_test, aggregations[key])

        # Création des dataloaders pour l'entraînement, la validation et le test.
        train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
        valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True)
        test_dataloaders[key] = DataLoader(test_dataset, batch_size=16, shuffle=True)

        # Initialisation des graines pour assurer la reproductibilité.
        set_seeds(42)

        # Définition du nom du répertoire pour sauvegarder le modèle.
        directory_name = 'model/{}_mlp'.format(aggregations[key].__name__)  

        # Création du modèle de perceptron multicouche pour chaque agrégation.
        models[key] = MultiLayerPerceptron(nlp_model.meta['vectors']['width'], hidden_size, nb_classes)

        # Création d'une expérience pour chaque modèle.
        experiments[key] = Experiment(directory_name, models[key], optimizer="Adam", task="classification")
        
        # Entraînement des modèles et stockage des résultats dans les journaux.
        loggings[key] = experiments[key].train(train_dataloader, valid_dataloader, epochs=30, disable_tensorboard=True)
    
    # Retour des modèles, journaux, expériences et dataloaders de test.
    return models, loggings, experiments, test_dataloaders


Le choix du nombre de couches cachées dépend du problème. Dans notre cas, on peut envisager qu'une taille de couche cachée entre 100 et 250 serait appropriée pour obtenir de bons résultats, en prenant en compte la taille de notre entrée (300) et de notre sortie (9). Si la couche cachée est excessivement grande, notre modèle pourrait ne pas être performant à cause du surapprentissage. Inversement, si la couche cachée est trop petite, notre réseau neuronal serait incapable de capturer suffisamment d'informations de nos embeddings de mots, conduisant à un sous-apprentissage. Dans mon cas, le nombre de couches cachées a été déterminé de manière itérative, en testant plusieurs valeurs dans l'intervalle estimé ci-dessus.

In [54]:
hidden_size=100

In [55]:
models, loggings, experiments, test_dataloaders = train(hidden_size=hidden_size)

Epoch:  1/30 Train steps: 155 Val steps: 34 58.43s loss: 1.553585 acc: 45.616162 fscore_macro: 0.158856 val_loss: 1.470433 val_acc: 45.009416 val_fscore_macro: 0.159505
Epoch 1: val_acc improved from -inf to 45.00942, saving file to model/average_embedding_mlp/checkpoint_epoch_1.ckpt
Epoch:  2/30 Train steps: 155 Val steps: 34 0.10s loss: 1.297341 acc: 54.585859 fscore_macro: 0.252139 val_loss: 1.280080 val_acc: 52.730697 val_fscore_macro: 0.265291
Epoch 2: val_acc improved from 45.00942 to 52.73070, saving file to model/average_embedding_mlp/checkpoint_epoch_2.ckpt
Epoch:  3/30 Train steps: 155 Val steps: 34 0.09s loss: 1.161069 acc: 59.232323 fscore_macro: 0.327440 val_loss: 1.144949 val_acc: 59.322034 val_fscore_macro: 0.363574
Epoch 3: val_acc improved from 52.73070 to 59.32203, saving file to model/average_embedding_mlp/checkpoint_epoch_3.ckpt
Epoch:  4/30 Train steps: 155 Val steps: 34 0.11s loss: 1.083474 acc: 61.737374 fscore_macro: 0.363161 val_loss: 1.080170 val_acc: 60.64030

## 6. Évaluation et analyse de résultats

In [58]:
def test(experiments = experiments):
    for key in aggregations.keys():
        print("Test for :", key, "\n")
        experiments[key].test(test_dataloaders[key])
        print("\n")

In [59]:
test()

Test for : average 

Found best checkpoint at epoch: 30
lr: 0.001, loss: 0.632674, acc: 77.8586, fscore_macro: 0.758985, val_loss: 0.946139, val_acc: 70.8098, val_fscore_macro: 0.675723
Loading checkpoint model/average_embedding_mlp/checkpoint_epoch_30.ckpt
Running test
Test steps: 34 0.02s test_loss: 0.946139 test_acc: 70.809793 test_fscore_macro: 0.675723          


Test for : maxpool 

Found best checkpoint at epoch: 30
lr: 0.001, loss: 1.43109, acc: 48.9293, fscore_macro: 0.278957, val_loss: 1.52163, val_acc: 45.1977, val_fscore_macro: 0.182609
Loading checkpoint model/maxpool_embedding_mlp/checkpoint_epoch_30.ckpt
Running test
Test steps: 34 0.01s test_loss: 1.521630 test_acc: 45.197740 test_fscore_macro: 0.182609          


Test for : minpool 

Found best checkpoint at epoch: 27
lr: 0.001, loss: 1.35599, acc: 51.1515, fscore_macro: 0.228557, val_loss: 1.41803, val_acc: 51.0358, val_fscore_macro: 0.229967
Loading checkpoint model/minpool_embedding_mlp/checkpoint_epoch_27.ckpt
Ru

## Analyse des resultats ##

On peut déduire que le modèle utilisant l'agrégation moyenne (Average Embedding) a nettement surpassé les modèles Maxpool et Minpool en termes de perte, de précision et de F-score. Cela suggère que l'approche d'agrégation moyenne est plus efficace pour ce problème spécifique, offrant un meilleur équilibre entre la capacité de modélisation et la généralisation par rapport aux approches Maxpool et Minpool.

Il est a noter que pour l'experience nous avons garde une taille de la couche cachee identique (100) sur chacune des approches pour quon puisse avoir une comparaison sur la meme base.